In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from matplotlib.ticker import FuncFormatter
from os.path import abspath
import re
import os
import matplotlib.image
from functools import partial
from IPython.display import Image
from sklearn import linear_model
from statistics import median 
import plotly.offline as py
import seaborn as sns
import plotly.figure_factory as ff
import pingouin as pg
import statsmodels.api as sm
import statsmodels.formula.api as smf
WEIGHT = 'WTPERFIN'

### Regression for Census Division & Incomes

In [ ]:
incCensus = pd.read_excel('datasets/UnitInStructure.xlsx', sheet_name = 'linearRegression',header =None,index_col=0)
incCensus.reset_index(inplace = True)

In [ ]:
incCensus.loc[14,0] = 162500

In [ ]:
arr = np.zeros(((len(incCensus.index)-1)*(len(incCensus.columns)-1), 3))
print(arr.shape)
incCensus.iloc[1,0]

In [ ]:
for i in range(len(incCensus.index)-1):
    for j in range(len(incCensus.columns)-1):
        arr[j+i*(len(incCensus.columns)-1)][0] = incCensus.iloc[i+1,0]
        arr[j+i*(len(incCensus.columns)-1)][1] = incCensus.iloc[0,j+1]
        arr[j+i*(len(incCensus.columns)-1)][2] = incCensus.iloc[i+1,j+1]

In [ ]:
arr
linPre = pd.DataFrame(arr,columns=['HHIncomes', 'CensusDivisions', 'MUDsShares'])
linPre
linPre_2 = linPre

In [ ]:
model = sm.OLS(linPre[['MUDsShares']], linPre[['HHIncomes','CensusDivisions']])
results = model.fit()
print(results.summary())

In [ ]:
# We have a categorical variable 'Census Divisions' which has 9 values, so we should only be using 8 dummy variables. 
# 'Pacific Division' becomes the reference category, in the case that the other dummy variables are all 0, 
# then by default the variable would be 'Pacific Division'.
division_list = ['NewEngland','MiddleAtlantic','EastNorthCentral','WestNorthCentral',\
                 'SouthAtlantic','EastSouthCentral','WestSouthCentral','Mountain']
for i, item in enumerate(division_list):
    linPre_2[item] = np.where(linPre_2['CensusDivisions']== i+1, 0, 1)

In [ ]:
X = sm.add_constant(linPre_2[['HHIncomes']+division_list])
model = sm.OLS(linPre_2[['MUDsShares']], X)
results_2 = model.fit()
#print(results_2.summary())
print(results_2.summary().as_latex())
with open('summary_no_interaction_terms.txt', 'w') as fh:
    fh.write(results_2.summary().as_latex())
#print('p-values are: \n',results_2.pvalues)

In [ ]:
# we also test the interaction terms hereby

model2 = smf.ols(formula = "MUDsShares ~ HHIncomes + NewEngland+MiddleAtlantic+EastNorthCentral+\
                 WestNorthCentral+SouthAtlantic+EastSouthCentral+WestSouthCentral+Mountain+\
                 HHIncomes:NewEngland+HHIncomes:MiddleAtlantic+HHIncomes:EastNorthCentral+HHIncomes:WestNorthCentral+\
                 HHIncomes:SouthAtlantic+HHIncomes:EastSouthCentral+HHIncomes:WestSouthCentral+HHIncomes:Mountain",data=linPre_2)
results_3 = model2.fit()
#print(results_3.summary())
#print('p-values are: \n',results_3.pvalues)
# do either
print(results_3.summary().as_latex())
with open('summary_with_interaction_terms.txt', 'w') as fh:
    fh.write(results_3.summary().as_latex())
# alternatively
# for table in results.summary().tables:
#     print(table.as_latex_tabular())

In [ ]:
pg.corr(x=linPre_2['HHIncomes'], y=linPre_2['NewEngland'])

In [ ]:
pg.pairwise_corr(linPre_2).sort_values(by=['p-unc'])[['X', 'Y', 'n', 'r', 'p-unc']].head()

In [ ]:
linPre_2

In [ ]:
linPre_2_2 = linPre_2.drop(columns='CensusDivisions')
linPre_2_2.head()

In [ ]:
linPre_2_2.corr().round(2)

In [ ]:
corrs = linPre_2_2.corr()
mask = np.zeros_like(corrs)
mask[np.triu_indices_from(mask)] = True
plt.figure(figsize=(20,10))
sns.heatmap(corrs, cmap='Spectral_r', mask=mask, square=True, vmin=-.8, vmax=.3)
plt.title('Correlation matrix')

### Pick out only one primary vehicle for each household to generate the sub-dataset of original trip data.

In [ ]:
# 'trippub.csv' can be downloaded from 
# Federal Highway Administration. (2017). 2017 National Household Travel Survey, U.S. Department of Transportation, Washington, DC. 
# Available online: https://nhts.ornl.gov.
trip17 = pd.read_csv('trippub.csv')
trip17.head()

In [ ]:
trip17_1pv = trip17.groupby(['HOUSEID','VEHID'], as_index=False)['VMT_MILE'].sum()

In [ ]:
trip17_process=trip17_1pv.groupby(['HOUSEID']).agg(list)

In [ ]:
trip17_process['freq_car_index']=trip17_process.agg(lambda x: x['VEHID'][np.argmax(x['VMT_MILE'])],axis=1)

In [ ]:
trip17_process=trip17_process.reset_index()

In [ ]:
trip17_process.head()

In [ ]:
trip17.head()

In [ ]:
trip17_final=trip17.merge(trip17_process[['HOUSEID','freq_car_index']],on=['HOUSEID'],how='left')

In [ ]:
trip17_final['selected_car']=trip17_final['freq_car_index']==trip17_final['VEHID']

In [ ]:
trip17_final[['freq_car_index','VEHID','selected_car']].head()

In [ ]:
trip17_final = trip17_final[trip17_final['selected_car'] == True]

In [ ]:
trip17_final.head()

In [ ]:
trip_inter = trip17_final.groupby(['HOUSEID','VEHID','HHFAMINC','CENSUS_D'], as_index=False)['VMT_MILE'].sum()

In [ ]:
for i, item in enumerate(division_list):
    trip_inter[item] = np.where(trip_inter['CENSUS_D']== i+1, 0, 1)
#trip_inter.head()

In [ ]:
trip_inter.loc[:, 'const'] = 1

In [ ]:
income = (5000, 12500, 20000, 30000, 40000, 62500, 87500,112500, 137500, 175000, 250000) 
#trip17_MUD_C.loc[:, 'MUDs_Predict'] = 0
trip_inter.loc[:, 'HHIncomes'] = 0
for i in range(11):
    trip_inter['HHIncomes'].mask(trip_inter['HHFAMINC'] == i+1, income[i], inplace=True)
trip_inter[trip_inter['HHIncomes'] == 0] = median(income)

In [ ]:
Xnew = [trip_inter['const'],trip_inter['HHIncomes'],trip_inter[division_list[0]],trip_inter[division_list[1]],\
            trip_inter[division_list[2]],trip_inter[division_list[3]], trip_inter[division_list[4]],\
            trip_inter[division_list[5]], trip_inter[division_list[6]], trip_inter[division_list[7]]]
   

In [ ]:
Xnew=np.array(Xnew)

In [ ]:
Xnew.shape

In [ ]:
X = sm.add_constant(Xnew)

In [ ]:
X.shape

In [ ]:
results_2.predict(X[:,132])

In [ ]:
trip_inter.loc[:, 'MUDs_Predict'] = 0
for i in range(len(trip_inter)):
    trip_inter.loc[i,'MUDs_Predict'] = results_2.predict(X[:,i])[0]

In [ ]:
## Predict corresponding MUDs share for trip takers
## merge MUD or others from AHS with the share of MUD in different CENSUS DIVISION

random_lists = []
for i in range(len(trip_inter)):
    if trip_inter.loc[i,'MUDs_Predict'] >= 1:
        #print(i)
        random_lists.append(1)
    elif trip_inter.loc[i,'MUDs_Predict'] > 0 :
        #print(i)
        random_lists.append(np.random.choice([1, 0], p=[trip_inter.loc[i,'MUDs_Predict'], 1-trip_inter.loc[i,'MUDs_Predict']]))
    else:
        random_lists.append(0)
trip_inter['MUDs_Predict_OW'] = random_lists


In [ ]:
random_lists = []
for i in range(len(trip_inter)):
    if trip_inter.loc[i,'MUDs_Predict'] >= 1:
        #print(i)
        random_lists.append(1)
    elif trip_inter.loc[i,'MUDs_Predict'] > 0 :
        #print(i)
        random_lists.append(np.random.choice([1, 0], p=[trip_inter.loc[i,'MUDs_Predict'], 1-trip_inter.loc[i,'MUDs_Predict']]))
    else:
        random_lists.append(0)
trip_inter['MUDs_Predict_OW_1'] = random_lists


In [ ]:
trip_inter['MUDs_Predict_OW'].value_counts()

### Multiple simulations to convince the robustness of the ranmized assignment

In [ ]:
###### Multiple times (9 times) of simulation with the randomized assignment
count = 2
while (count < 10):
    random_lists = []
    for i in range(len(trip_inter)):
        if trip_inter.loc[i,'MUDs_Predict'] >= 1:
            #print(i)
            random_lists.append(1)
        elif trip_inter.loc[i,'MUDs_Predict'] > 0 :
            #print(i)
            random_lists.append(np.random.choice([1, 0], p=[trip_inter.loc[i,'MUDs_Predict'], 1-trip_inter.loc[i,'MUDs_Predict']]))
        else:
            random_lists.append(0)
    trip_inter['MUDs_Predict_OW_'+str(count)] = random_lists
    count += 1
    

In [ ]:
MUDs_no_lists = []
MUDs_no_lists.append(trip_inter['MUDs_Predict_OW'].sum())
for count in range(1,10):
    MUDs_no_lists.append(trip_inter['MUDs_Predict_OW_'+str(count)].sum())

In [ ]:
MUDs_no_lists

In [ ]:
###### (Expanding) Multiple times (90 times) of simulation with the randomized assignment
count = 10
while (count < 100):
    random_lists = []
    for i in range(len(trip_inter)):
        if trip_inter.loc[i,'MUDs_Predict'] >= 1:
            #print(i)
            random_lists.append(1)
        elif trip_inter.loc[i,'MUDs_Predict'] > 0 :
            #print(i)
            random_lists.append(np.random.choice([1, 0], p=[trip_inter.loc[i,'MUDs_Predict'], 1-trip_inter.loc[i,'MUDs_Predict']]))
        else:
            random_lists.append(0)
    trip_inter['MUDs_Predict_OW_'+str(count)] = random_lists
    MUDs_no_lists.append(trip_inter['MUDs_Predict_OW_'+str(count)].sum())
    count += 1
    

In [ ]:
trip_inter.head()

In [ ]:
# the first simulation
simu = trip_inter.loc[:,'MUDs_Predict_OW':'MUDs_Predict_OW_1']

In [ ]:
# 100 simulation
simu = trip_inter.loc[:,'MUDs_Predict_OW':'MUDs_Predict_OW_99']

In [ ]:
# simu.to_csv('new_simulation_result_income_162500.csv',index=False)

### check each division has approximate MUDs share from the simulation

In [ ]:
trip17_pred=trip17_final.merge(simu,on=['HOUSEID'],how='left')

In [ ]:
trip17_pred.head()

In [ ]:
division_list_9 = ['New England','Middle Atlantic','East North Central','West North Central',\
                 'South Atlantic','East South Central','West South Central','Mountain','Pacific']

In [ ]:
trip17_pred_uni = trip17_pred.groupby(['HOUSEID','CENSUS_D'], as_index=False)['MUDs_Predict_OW'].mean()

In [ ]:
division = trip17_pred_uni[trip17_pred_uni['CENSUS_D'] == 0+3]
division['MUDs_Predict_OW'].value_counts()

In [ ]:
MUD_share = []
for i in range(9):
    division = trip17_pred_uni[trip17_pred_uni['CENSUS_D'] == i+1]
    temp = division['MUDs_Predict_OW'].value_counts().tolist()[1]/sum(division['MUDs_Predict_OW'].value_counts().tolist())
    temp = round(temp, 3)
    MUD_share.append(temp)
    print('MUDs share of',division_list_9[i],'division is: ',MUD_share[i])

In [ ]:
fig, ax = plt.subplots(figsize=(8,5)) 
ax.bar(division_list_9,MUD_share)
ax.set_title('MUDs Share per Census Division')

for tick in ax.get_xticklabels():
    tick.set_rotation(45)
#ax.set_rotation(rotation=45)